# ようこそ

maDMPへようこそ。ここでは、DMPに入力いただいた研究分野等情報に基づき、ワークフローを最適化します。  
以下のセルを実行することで、ワークフローを生成し、DMP情報を基に最適化します。

In [ ]:
# DMP情報
field = '%v'

In [ ]:
# ワークフローテンプレート取得
clone_url = 'https://github.com/ivis-kuwata/workflow-template'+'.git'
workflow_path = '/home/jovyan/WORKFLOW'
!git clone $clone_url $workflow_path
!rm -rf $workflow_path/.git

In [ ]:
# dmp.jsonに"fields"プロパティがある想定
import os
import glob
import shutil

path_flows = os.path.join('WORKFLOW', 'FLOW')

templates = glob.glob(os.path.join(path_flows, '**'), recursive=True)

# 選択外の分野のセクション群を削除
for tmpl in templates:
    file = os.path.basename(tmpl)
    if not os.path.isdir(tmpl) and os.path.splitext(file)[1] == '.ipynb':
        if 'base_' not in file and field not in file:
            os.remove(tmpl)
            
print('最適化処理が完了しました。次の手順へお進みください。')

# ワークフロー全体の関連図と目次を生成する

次に、最適化したワークフロー全体が俯瞰できる図を生成します。  
以下を実行することで、システムがワークフロー関連図、および目次を作成します。

◆◆◆開発メモ◆◆◆

*.diagに分野固有のセクションはまだ反映されていない？

In [ ]:
# 必要パッケージのインストール
!pip install lxml==4.7.1
!pip install blockdiag==3.0.0
!pip install nbformat==5.1.3

# 日本語対応フォントのインストール
!mkdir ~/.fonts
!cd ~/.fonts

font_deb = 'fonts-ipafont-gothic_00303-18ubuntu1_all.deb'
!wget -P ~/.fonts http://archive.ubuntu.com/ubuntu/pool/universe/f/fonts-ipafont/$font_deb
!dpkg-deb -x ~/.fonts/$font_deb ~/.fonts
!cp ~/.fonts/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf ~/.fonts/ipag.ttf

In [ ]:
import nbformat

flow_path = 'WORKFLOW/FLOW/base_FLOW.ipynb'
nb = nbformat.read(flow_path, 4)

title = """## ワークフロー図を作成する

以下のセルを実行してワークフロー図を作成できます。
各ワークフローを実行するにはそれぞれのリンクをクリックしてください。"""

code = """from IPython.display import SVG
!cd ~/
%%run /home/jovyan/WORKFLOW/scripts/nb_utils.py
SVG(filename=generate_svg_diag(diag='WORKFLOW/images/notebooks.diag'))"""

nb["cells"] += [
    nbformat.v4.new_markdown_cell(title),
    nbformat.v4.new_code_cell(code)
]

with open(flow_path, "w") as f:
    nbformat.write(nb, f)
